In [1]:
from nnsight import LanguageModel

In [2]:
model = LanguageModel("openai-community/gpt2", device_map="auto")
print(model)

/home/ubuntu/qwen2-05b-instruct/monosemanticity/experiments/.experiments/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): WrapperModule()
)


In [12]:
with model.trace("The Eiffel Tower is in the city of"):
    model.transformer.h[-1].mlp.output[0][:] = 0

    token_ids = model.lm_head.output.argmax(dim=-1).save()

print("token ids:", token_ids)
print("prediction", model.tokenizer.decode(token_ids[0][-1]))

token ids: tensor([[ 262,   12,  417, 8765,   11,  257,  262, 3504,  338, 3576]],
       device='cuda:0')
prediction  London


In [18]:
with model.trace() as tracer:
    with tracer.invoke("The Eiffel Tower is in the city of"):
        model.transformer.h[-1].mlp.output[0][:] = 0
        token_ids_intervention = model.lm_head.output.argmax(dim=-1).save()

    with tracer.invoke("The Eiffel Tower is in the city of"):
        token_ids_original = model.lm_head.output.argmax(dim=-1).save()

print("token ids original:", token_ids_original)
print("token ids intervention:", token_ids_intervention)

print("original prediction", model.tokenizer.decode(token_ids_original[0][-1]))
print("intervention prediction", model.tokenizer.decode(token_ids_intervention[0][-1]))

token ids original: tensor([[ 198,   12,  417, 8765,  318,  257,  262, 3504, 7372, 6342]],
       device='cuda:0')
token ids intervention: tensor([[ 262,   12,  417, 8765,   11,  257,  262, 3504,  338, 3576]],
       device='cuda:0')
original prediction  Paris
intervention prediction  London
